### Problem Statement

Included under Milestone 3 — "Project Question and Model"

### Explore and Visualize Data

Included in EDA.ipynb

### Baseline Model

In [1]:
# Imports
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer, KNNImputer
from scipy.stats import entropy
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime, timedelta
import re
import matplotlib.ticker as mtick

In [2]:
df = pd.read_csv('processed_data/pa_data_all_elections_sample.csv')
df.head()
print(df.columns)


Index(['DOB', 'Registration Date', 'Voter Status', 'Status Change Date',
       'Date Last Changed', 'Gender M', 'Gender F', 'Gender U', 'Party D',
       'Party R',
       ...
       'Election 01/03/2006 Party R', 'Election 01/03/2006 Party I',
       'Election 01/03/2006 Voted', 'Election 01/03/2006 By Mail',
       'Election 09/09/2008 Presence', 'Election 09/09/2008 Party D',
       'Election 09/09/2008 Party R', 'Election 09/09/2008 Party I',
       'Election 09/09/2008 Voted', 'Election 09/09/2008 By Mail'],
      dtype='object', length=511)


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

# Define election_columns
election_columns = [col for col in df.columns if col.startswith('Election')]

# Assuming df is your DataFrame containing the election columns
X = df.drop(election_columns, axis=1)
all_residuals = []
all_predictions = []

for election_col in election_columns:
    y = df[election_col]

    # Impute NaN values in y using mean imputation
    imputer = SimpleImputer(strategy='mean')
    y_imputed = imputer.fit_transform(y.values.reshape(-1, 1)).ravel()

    # Splitting the data
    X_train, X_temp, y_train, y_temp = train_test_split(X, y_imputed, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    test_predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, test_predictions)
    print(f"Mean Squared Error on testing set for {election_col}: {mse}")

    # OLS model
    model_ols = sm.OLS(y_train, sm.add_constant(X_train)).fit()
    predictions = model_ols.predict(sm.add_constant(X_train))
    residuals = y_train - predictions

    # Store residuals and predictions for the final plot
    all_residuals.extend(residuals)
    all_predictions.extend(predictions)

# Plotting the combined residuals vs. fitted values for all election columns
sns.scatterplot(x=all_predictions, y=all_residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Combined Residuals vs. Predicted Values for All Election Columns')
plt.show()


Mean Squared Error on testing set for Election 04/27/2004 Presence: 0.224470144746691
Mean Squared Error on testing set for Election 04/27/2004 Party D: 0.013678954155946266
Mean Squared Error on testing set for Election 04/27/2004 Party R: 0.0224749521391153
Mean Squared Error on testing set for Election 04/27/2004 Party I: 0.03254977728793222
Mean Squared Error on testing set for Election 04/27/2004 Voted: 0.03217285104483569
Mean Squared Error on testing set for Election 04/27/2004 By Mail: 0.0004179132312051789
Mean Squared Error on testing set for Election 05/15/2018 Presence: 0.0
Mean Squared Error on testing set for Election 05/15/2018 Party D: 0.06478847473996861
Mean Squared Error on testing set for Election 05/15/2018 Party R: 0.057908854327471435
Mean Squared Error on testing set for Election 05/15/2018 Party I: 0.10920893299381042
Mean Squared Error on testing set for Election 05/15/2018 Voted: 0.10956990491440104
Mean Squared Error on testing set for Election 05/15/2018 By

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

# Define election_columns
election_columns = [col for col in df.columns if 'Election' in col and 'Voted' in col]

# List to store residuals and predictions from all models
all_residuals = []
all_predictions = []

# Assuming df is your DataFrame containing the election columns
for target_col in election_columns:
    X = df.drop(target_col, axis=1)
    y = df[target_col]

    # Impute NaN values in X and y using mean imputation
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    y_imputed = imputer.fit_transform(y.values.reshape(-1, 1)).ravel()

    # Fit OLS model on entire dataset (excluding the held-out column)
    final_model = sm.OLS(y_imputed, sm.add_constant(X_imputed))
    final_results = final_model.fit()

    # Generate predictions and residuals
    predictions = final_results.predict(sm.add_constant(X_imputed))
    residuals = y_imputed - predictions

    # Store results for the final plot
    all_residuals.extend(residuals)
    all_predictions.extend(predictions)

# Plotting the combined residuals vs. fitted values for all election columns
sns.scatterplot(x=all_predictions, y=all_residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Combined Residuals vs. Predicted Values for All Election Columns')
plt.show()


### Results Interpretation

### Final Model Pipeline